# EventIO Tutorial

The need to perform several actions concurrently arises frequently in microcontroller applications. Sensor outputs need to be read, motors and LEDs controlled, and buttons or touchpads checked for user input.

While it is possible in principle to perform all these actions sequentially in a loop, except in simple applications such solutions are difficult to get correct and even harder to maintain when additional capabilities are introduced.

This tutorial introduces `eventio`, a *partial* implementation of `curio`, a [library for concurrent processing](https://curio.readthedocs.io). 

## Blinking one LED

Let's start the tour with "blink", the proverbial "Hello World" application for microcontrollers (change the name of the LEDs to match your board).

In [45]:
%serialconnect "/dev/cu.SLAB_USBtoUART"

serial exception on close write failed: [Errno 6] Device not configured
Found serial ports: /dev/cu.SLAB_USBtoUART, /dev/cu.Bluetooth-Incoming-Port, /dev/cu.MALS, /dev/cu.SOC 
Connecting to --port=/dev/cu.SLAB_USBtoUART --baud=115200 
Ready.


In [46]:
from board import LED as LED_R
from board import A19 as LED_G
from board import A20 as LED_B
from machine import Pin
import time

def blink(color, pin, period):
    p = Pin(pin, mode=Pin.INOUT)
    for i in range(3):
        time.sleep(period/2)
        p.value(not p.value())
    
blink("red", LED_R, 0.5)

The code blinks the LED the specified number of times and then stops. Nothing unusual.

## Blinking multiple LEDs simulataneously

Now suppose we wanted to blink several LEDs at individual rates, say 0.7ms and 0.4ms periods. Simply calling calling `blink` twice with different parameters won't produce the desired effect:

In [47]:
blink("red",   LED_R, 0.7)
blink("green", LED_G, 0.4)

This blinks the red light followed by the green light, one after the other. To blink them simultaneously but at different rates, modify `blink` as follows:

In [48]:
import eventio

async def blink(color, pin, period):
    p = Pin(pin, mode=Pin.INOUT)
    for i in range(10):
        await eventio.sleep(period/2)
        p.value(not p.value())

The differences are importing `eventio` and peceeding the function definition with `async`. This marks it as a `coroutine`, a piece of code that can run concurrently with other operations. 

The call to `time.sleep` is replaced by `await eventio.sleep`. The keyword `await` tells the Python interpreter to perform other activity, turing on and off other LEDs in this case, simultaneously.

The following code blinkes the red, green, and blue LED simultaneously:

In [49]:
async def main():
    r = await eventio.spawn(blink, "red  ", LED_R, 0.7)
    g = await eventio.spawn(blink, "green", LED_G, 0.4)
    b = await eventio.spawn(blink, "blue ", LED_B, 0.5)
    
eventio.run(main)

.

The calls to `eventio.spawn` create three separate coroutines which all run in parallel. The call to `eventio.run` starts the process.

In addition to the visual confirmation, we can add printed output to convince us that the coroutines indeed run concurrently:

In [50]:
%rebootdevice
%serialconnect "/dev/cu.SLAB_USBtoUART"

normal repl mode not detected b'\r\nMicroPython ESP32_LoBo_v3.2.24 - 2018-09-06 on Huzzah32 with ESP32\r\nType "help()" for more information.\r\n>>> \r\n>>> \r\nESP32: soft reboot\r\nets Jun  8 2016 00:22:57\r\n\r\nrst:0xc (SW_CPU_RESET),boot:0x13 (SPI_FAST_FLASH_BOOT)\r\nconfigsip: 0, SPIWP:0xee\r\nclk_drv:0x00,q_drv:0x00,d_drv:0x00,cs0_drv:0x00,hd_drv:0x00,wp_drv:0x00\r\nmode:DIO, clock div:2\r\nload:0x3fff0018,len:4\r\nload:0x3fff001c,len:4916\r\nload:0x40078000,len:8652\r\nload:0x40080000,len:5748\r\nentry 0x400802dc\r\n'
not command mode

Internal FS (SPIFFS): Mounted on partition 'internalfs' [size: 2621440; Flash address: 0x180000]
----------------
Filesystem size: 2402304 B
           Used: 512 B
           Free: 2401792 B
----------------
MicroPython ESP32_LoBo_v3.2.24 - 2018-09-06 on Huzzah32 with ESP32
Type "help()" for more information.
>>>[reboot detected 0]repl is in normal command mode
[\r\x03\x03] b'\r\n>>> '
[\r\x01] b'\r\n>>> \r\nraw REPL; CTRL-B to exit\r\n>' Found s

In [51]:
from board import LED as LED_R
from board import A19 as LED_G
from board import A20 as LED_B
from machine import Pin
import eventio

chrono = eventio.Chronometer()

async def blink(color, pin, period):
    global chrono
    p = Pin(pin, mode=Pin.INOUT)
    last_time = 0
    for i in range(10):
        await eventio.sleep(period/2)
        p.value(not p.value())
        elapsed = chrono.elapsed_time
        delta = elapsed-last_time
        extra_delay = delta - period/2
        print("{} at {:6.0f} ms, {:4.0f} ms since last call, {:6.0f} ms extra".format( \
            color, 1000*elapsed, 1000*delta, 1000*extra_delay))
        last_time = chrono.elapsed_time
    print("{} is done".format(color))
    
async def main():
    r = await eventio.spawn(blink, "red  ", LED_R, 0.3)
    g = await eventio.spawn(blink, "green", LED_G, 0.7)
    b = await eventio.spawn(blink, "blue ", LED_B, 0.5)
    print("all LEDs blinking ...")
    await g.join()   # wait for coroutine g to terminate
    # print some statistics
    kernel = await eventio.get_kernel()
    print("Program ran for {} seconds with {} % CPU utilization".format(
        kernel.uptime(), kernel.load_average()
    ))
    
eventio.run(main)

all LEDs blinking ...
red   at    160 ms,  160 ms since last call,     10 ms extra
blue  at    272 ms,  272 ms since last call,     22 ms extra
red   at    320 ms,  153 ms since last call,      3 ms extra
green at    366 ms,  366 ms since last call,     16 ms extra
red   at    480 ms,  154 ms since last call,      4 ms extra
blue  at    531 ms,  253 ms since last call,      3 ms extra
red   at    640 ms,  154 ms since last call,      4 ms extra
green at    725 ms,  353 ms since last call,      3 ms extra
blue  at    791 ms,  253 ms since last call,      3 ms extra
red   at    802 ms,  156 ms since last call,      6 ms extra
red   at    962 ms,  154 ms since last call,      4 ms extra
blue  at   1052 ms,  254 ms since last call,      4 ms extra
green at   1084 ms,  353 ms since last call,      3 ms extra
red   at   1122 ms,  154 ms since last call,      4 ms extra
red   at   1281 ms,  153 ms since last call,      3 ms extra
blue  at   1312 ms,  254 ms since last call,      4 ms extra
re

The printed output confirms that the coroutines indeed run simultataneously. Also shown is the time (in milli-seconds) at which the LED is turned on or off. Analyzing them you will notice that each LED is called at multiples of half it's period, plus a small "processing delay". Try to program all of this in a single loop ... And this is just for blinking LEDs; many real applications have more complex requirements!

Unlike calling `time.sleep`, which litterally instructs the processor to do `nothing` except dissipate power and heat up the planet, a call to `eventio.sleep` checks for other work to be done, which in this example amounts to checking if one of the `blink` coroutines is ready to run. If no coroutine is ready to execute, `eventio.sleep` powers the processor down, which reduces the supply current by more than three orders-of-magnitude. 

Indeed, while the entire program ran for several seconds, the processor was in the high-power `awake` state less than 2 percent of the time, corresponding to a 50 times reduction of the energy used. Given the populatity of programs that blink LEDs, `eventio` has the potential of making a significant contribution towards saving the planet!

## Input

In the current implementation the LEDs blink for a fixed number of times. The code below stops when button `MODE` (adapt to your board) is pressed.

In [ ]:
%serialconnect "/dev/cu.SLAB_USBtoUART"

from board import LED as LED_R
from board import A19 as LED_G
from board import A20 as LED_B
from board import A5 as MODE
from machine import Pin
import eventio

sw1_event = eventio.PinEvent(MODE)

async def blink(color, pin, period):
    p = Pin(pin, mode=Pin.INOUT)
    try:
        while True:
            await eventio.sleep(period/2)
            p.value(not p.value())
    except eventio.CancelledError:
        print("{} cancelled".format(color))
        p.value(False)   # LED off
        raise

async def main():
    r = await eventio.spawn(blink, "red  ", LED_R, 0.3)
    g = await eventio.spawn(blink, "green", LED_G, 0.7)
    b = await eventio.spawn(blink, "blue ", LED_B, 0.5)
    print("All LEDs blinking ... press button to stop!")
    await sw1_event.wait()
    print("Button pressed! Cancelling blinkers ...")
    await r.cancel()
    await g.cancel()
    await b.cancel()
    # print some statistics
    kernel = await eventio.get_kernel()
    print("Program ran for {:.1f} seconds with {:.1f} % CPU utilization".format(
        kernel.uptime(), kernel.load_average()
    ))
    
eventio.run(main)

Found serial ports: /dev/cu.SLAB_USBtoUART, /dev/cu.Bluetooth-Incoming-Port, /dev/cu.MALS, /dev/cu.SOC 
Connecting to --port=/dev/cu.SLAB_USBtoUART --baud=115200 
Ready.
All LEDs blinking ... press button to stop!


## Concurrency and Parallel Processing

Although it appears as if coroutines run in parallel, only one is active at any given time. This is similar to other computers (laptops, desktops, etc) which give the appearance of running several programs simultaneously but are in fact switching rapidly between different tasks (multi-core processors do run more than one program concurrently). If the processor alternates rapidly e.g. between displaying a movie and a spreadsheet, it appears as if both were running simultaneously, just as the different colored LEDs are blinking concurrently.

A big difference between the parallel processing of laptop and desktop computers and `eventio` is the approach taken to switching between tasks. The former use a technique known as **"preemptive multitasking"**. In this case, the operating system (e.g. Linux or Windows) uses a timer to rapidly (e.g. every 50ms) switch between tasks: The currently running program is temporarily suspended, it's state saved, and a different task is permitted to execute. This process repeats, with each task getting a turn. If a task is current blocked, e.g. waiting for input, it is skipped until it is ready again.

With preemptive multitasking, tasks are not aware when execution is interrupted and have no control over when this happens. By contrast, `eventio` uses a different form of task switching called **"cooperative multitasking"**. In this scenario, task have full control over when they relinquish the CPU, namely when calling `await`. In other words, a coroutine can be assured that no other task will interrupt it until it signals its "consent" with `await`. The duration of the suspension depends on the statement called and the tasks that are waiting. E.g. `await eventio.sleep(1)` suspends the task for at least one second, but could also suspend it for longer if other tasks need the CPU.

Preemptive and cooperative multitasking have both advantages and drawbacks, and which solution is better depends on the application. For small microcontrollers with limited memory and processing speed, cooperative multitasking is attractive since it generally uses less memory. More significantly, however, is the fact that task switching happens only at well defined instances marked with the keyword `await`.

One of the most challenging aspect of parallel processing is ensuring proper use of shared resources. For example, imagine you want to read the value from a sensor over `I2C`. The driver first sends the address of the sensor to get its attention. Then asks the sensor send its value over the bus.

If this sequence is interrupted at an unfortunate moment, by another task, the program may not function as intended. For example, if after addressing the sensor a different task gets control and adresses a different sensor, once control returns to the first task the request to read the sensor data goes to the wrong sensor producing the wrong or no answer.

Such errors are difficult to detect, as they may occur only infrequently and are difficult to reproduce. With cooperative multitasking a task can be assured that it will not be interrupted unless calling `await`. While this simple solution does not address all situations, it substantially reduces the opportunities for errors in concurrent programs.

The main drawback of cooperative multitasking is that a (rouge) task can prevent all others from running. E.g. if in the above example, the call to `eventio.sleep` is replaced by `time.sleep` (and the `await` keyword removed), the first coroutine will run exclusively until it finishes. In a desktop environment this obviously would not be practical - a simple programming mistake could freeze the entire computer. On a microcontroller, however, the entire application is usually under control of a single user who can make sure that this does not happen.

## Caveats

In addition to the situation where a coroutine fails to call `await`, either due to a programming error or maliciously, thus preventing other tasks to run, there are a few other errors to avoid.

The first, and most vexing (the kind where everything looks right but still refuses to "work") is forgetting the `await` keyword, as in the example below:

In [43]:
from board import LED as LED_R
from machine import Pin
import eventio

async def blink(color, pin, period):
    p = Pin(pin, mode=Pin.INOUT)
    # ERROR: no await!
    eventio.sleep(period/2)
    p.value(not p.value())
    
eventio.run(blink, "red", LED_R, 1)
print("Done!")

Done!


This program runs very quickly without error, but `eventio.sleep(period/2)` does not "sleep", hence resulting in the LED blinking very rapidly (faster than discernible).

The result of `eventio.sleep(1)` is a generator (or coroutine in some versions of Python):

In [44]:
print(eventio.sleep(1))

<generator object 'sleep' at 3ffd0f90>


It is the `await` keyword that instructs the interpreter to actually perform the function. Unfortunately in Python the statements with and without `await` are syntactically correct, so watch out!

The other common error is calling `time.sleep` instead of `eventio.sleep`. The former, with `await`, gives an error and is thus easy to spot. But without `await`, `time.sleep` is of course a valid call, except that it blocks the CPU, preventing other tasks from running. 

If a concurrent program does not function as expected, these are some of the things to check!